# Evaluation of Multi-Agent approach using an orchestrator

In [5]:
import sys
import os
sys.path.append("/Users/i551965/Documents/dev/Uni/LLMsAgents/LLMsAgents-TextToSQL/src/")
sys.path.append("/Users/i551965/Documents/dev/Uni/LLMsAgents/LLMsAgents-TextToSQL/src/agents")
sys.path.append("/Users/i551965/Documents/dev/Uni/LLMsAgents/LLMsAgents-TextToSQL/src/prompt_templates")
from dotenv import load_dotenv

sys.path.insert(0, os.path.abspath("../src"))
print(sys.path)

load_dotenv()
from approach_evaluator import ApproachEvaluator
from orchestrator_agent import create_sql_from_natural_text, visualize_graph
visualize_graph()

['/Users/i551965/Documents/dev/Uni/LLMsAgents/LLMsAgents-TextToSQL/src', '/Users/i551965/Documents/dev/Uni/LLMsAgents/LLMsAgents-TextToSQL/src', '/opt/homebrew/Cellar/python@3.13/3.13.0_1/Frameworks/Python.framework/Versions/3.13/lib/python313.zip', '/opt/homebrew/Cellar/python@3.13/3.13.0_1/Frameworks/Python.framework/Versions/3.13/lib/python3.13', '/opt/homebrew/Cellar/python@3.13/3.13.0_1/Frameworks/Python.framework/Versions/3.13/lib/python3.13/lib-dynload', '', '/Users/i551965/Documents/dev/Uni/LLMsAgents/LLMsAgents-TextToSQL/.venv/lib/python3.13/site-packages', '/Users/i551965/Documents/dev/Uni/LLMsAgents/LLMsAgents-TextToSQL/src/', '/Users/i551965/Documents/dev/Uni/LLMsAgents/LLMsAgents-TextToSQL/src/agents', '/Users/i551965/Documents/dev/Uni/LLMsAgents/LLMsAgents-TextToSQL/src/prompt_templates', '/Users/i551965/Documents/dev/Uni/LLMsAgents/LLMsAgents-TextToSQL/src/', '/Users/i551965/Documents/dev/Uni/LLMsAgents/LLMsAgents-TextToSQL/src/agents', '/Users/i551965/Documents/dev/Uni/

In [ ]:
import pandas as pd

data_dir = os.getenv("DATA_DIRECTORY")
sql_input_file = pd.read_csv("../sample/dev.csv").sample(1)

# Config is just for logging purposes and does not affect the run
run_config = {
    "k_shot_prompting_creator_agent": 2,
    "k_shot_prompting_fixer_agent": 1,
    "k_shot_prompting_feedback_agent": 2,
    "k_shot_prompting_select_relevant_database": 0,
    "k_shot_prompting_select_relevant_tables": 2,
    "llm_orchestrator": "gpt4o",
    "llm_fixer": "gpt4o",
    "llm_feedback": "gpt4o",
    "llm_temperature": 0
}

evaluator = ApproachEvaluator(
    run_config=run_config,
    question_ids=sql_input_file["question_id"],
    approach=create_sql_from_natural_text
)
evaluator.evaluate(verbose=True)

Creator agent: Feedback: 
Creator agent: Feedback: [{"query_result": "[]", "is_correct": false, "feedback": "The query did not return any results, which indicates an issue with the query. The field names should not be enclosed in double quotes. Also, check if the 'type' field is correctly filtered.", "updated_query": "SELECT Label FROM trans WHERE trans_id = 'TR001_10_11' AND type = 'transaction';"}]
Creator agent: Feedback: [{"query_result": "[]", "is_correct": false, "feedback": "The query did not return any results, which indicates an issue with the query. The field names should not be enclosed in double quotes. Also, check if the 'type' field is correctly filtered.", "updated_query": "SELECT Label FROM trans WHERE trans_id = 'TR001_10_11' AND type = 'transaction';"}, {"Natural Language Prompt/ Original Question": "What is the label for bond ID TR001_10_11?", "Syntactically Incorrect SQL Query": "What is the label for bond ID TR001_10_11?", "Error Message": "Error executing query: n

{'question_id': [279],
 'predicted_query': ["SELECT k_symbol FROM trans WHERE trans_id = 'TR001_10_11';"],
 'predicted_database': ['financial'],
 'feedbacks': [[{'query_result': '[]',
    'is_correct': False,
    'feedback': "The query did not return any results, which indicates an issue with the query. The field names should not be enclosed in double quotes. Also, check if the 'type' field is correctly filtered.",
    'updated_query': "SELECT Label FROM trans WHERE trans_id = 'TR001_10_11' AND type = 'transaction';"},
   {'Natural Language Prompt/ Original Question': 'What is the label for bond ID TR001_10_11?',
    'Syntactically Incorrect SQL Query': 'What is the label for bond ID TR001_10_11?',
    'Error Message': 'Error executing query: no such column: Label',
    'Error Position': "Column name 'Label' is not recognized in the query.",
    'Specific Error': "The column name 'Label' is not present in any of the tables in the database.",
    'Advise': "Check the column names in the

In [9]:
import pandas as pd
run = pd.read_csv("./results/results_full_run_291124/results.csv")

run.head()

,input_texts,predicted_query,predicted_database,target_queries,is_correct
0,Among the cards with converted mana cost highe...,SELECT COUNT(*) FROM cards WHERE convertedMana...,card_games,SELECT SUM(CASE WHEN T1.power LIKE '*' OR T1.p...,0
1,Calculate the average home team goal in the 20...,SELECT AVG(home_team_goal) FROM Match WHERE se...,european_football_2,SELECT CAST(SUM(t2.home_team_goal) AS REAL) / ...,1
2,Which college do most of the members go to?,SQL\r\n\r\nSELECT college\r\nFROM major\r\nWHE...,student_club,SELECT T2.college FROM member AS T1 INNER JOIN...,0
3,Calculate the percentage of molecules containi...,SELECT (CAST(COUNT(DISTINCT(molecule_id)) AS R...,toxicology,SELECT CAST(COUNT(CASE WHEN T1.element = 'h' A...,0
4,"Provide the related post title of ""How to tell...","SELECT ""Post Title"" FROM 'thrombosis_predictio...",thrombosis_prediction,SELECT T3.Title FROM postLinks AS T1 INNER JOI...,0


In [10]:
from approach_evaluator import valid_efficiency_score

ves = valid_efficiency_score(run)

print(f"Valid Efficiency Score: {ves}")

Valid Efficiency Score: 0.20436181786987834
